In [6]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import torch
import torch.optim as optim
import warnings
import datetime
from sklearn import preprocessing
warnings.filterwarnings("ignore")
%matplotlib inline

In [49]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df_hoildays = pd.read_csv('holidays_events.csv')
df_stores = pd.read_csv('stores.csv')
df_transactions = pd.read_csv('transactions.csv')
df_oil = pd.read_csv('oil.csv')
real = pd.read_csv('sample_submission.csv')

In [52]:
def RMSLE(x,y):
    x = np.log1p(x)
    y = np.log1p(y)

# 计算均方误差（MSE）
    mse = np.mean(( x- y) ** 2)

# 计算RMSLE
    rmsle = np.sqrt(mse)
    print("RMSLE:", rmsle)
    return rmsle

## oil nan processing

In [50]:
for i in range(len(df_oil['dcoilwtico'])):
    is_na = pd.isna(df_oil['dcoilwtico'][i])
    
    # 处理第一个元素的NaN值
    if i == 0 and is_na:
        df_oil['dcoilwtico'][i] = df_oil['dcoilwtico'][i+1]
    
    # 处理中间连续的NaN值
    if i > 0 and is_na:
        j = 1
        while pd.isna(df_oil['dcoilwtico'][i-j]) or pd.isna(df_oil['dcoilwtico'][i+j]) :#这一步while很巧妙
            j += 1
        df_oil['dcoilwtico'][i] = (df_oil['dcoilwtico'][i-j] + df_oil['dcoilwtico'][i+j]) / 2

    # 处理最后一个元素的NaN值
    if i == len(df_oil['dcoilwtico']) - 1 and is_na:
        df_oil['dcoilwtico'][i] = df_oil['dcoilwtico'][i-1]


In [56]:
df_train.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013/1/1,1,AUTOMOTIVE,0.0,0
1,1,2013/1/1,1,BABY CARE,0.0,0
2,2,2013/1/1,1,BEAUTY,0.0,0
3,3,2013/1/1,1,BEVERAGES,0.0,0
4,4,2013/1/1,1,BOOKS,0.0,0


In [57]:
df_oil.head()

,date,dcoilwtico
0,2013-01-01,93.14
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20


In [60]:
df_hoildays.head()

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False


In [59]:
df_stores.head()

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [58]:
df_transactions.head()

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922


date             object
store_nbr         int64
family           object
sales           float64
onpromotion       int64
dcoilwtico      float64
type_x           object
locale           object
locale_name      object
description      object
transferred      object
city             object
state            object
type_y           object
cluster           int64
transactions    float64
dtype: object

In [83]:
def merge(data1, data2, data3):
    list_datetime = [data1, data2, data3]
    for i in range(len(list_datetime)):
        list_datetime[i] = pd.to_datetime(list_datetime[i]['date'], format='%Y-%m-%d')
        
        list_datetime[i] = list_datetime[i].to_frame().set_index('date')
    
    result = list_datetime[0]
    for i in range(len(list_datetime)):
        result = result.merge(list_datetime[i], left_index=True, right_index=True, how='left')
    
    result.reset_index(inplace=True)
    
    result = result.set_index('store_nbr')
    data4 = data4.set_index('store_nbr')
    result = result.merge(data4, left_index=True, right_index=True, how='left')
    
    result.reset_index(inplace=True)
    
    result = result.set_index(['store_nbr', 'date'])
    data5 = data5.set_index(['store_nbr', 'date'])
    
    result = result.merge(data5, left_index=True, right_index=True, how='left')
    
    result.reset_index(inplace=True)
    result.set_index('id', inplace=True)
    
    return result


In [84]:
train = merge1(df_train,df_oil,df_hoildays)

MemoryError: Unable to allocate 13.9 GiB for an array with shape (1867784193,) and data type int64

In [69]:
tl=[df_train,df_oil,df_hoildays,df_stores,df_transactions]
tl[1]

,date,dcoilwtico
0,2013-01-01,93.14
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20
...,...,...
1213,2017-08-25,47.65
1214,2017-08-28,46.40
1215,2017-08-29,46.46
1216,2017-08-30,45.96


In [77]:
for dataset in tl:
    for column in dataset.select_dtypes(include=['int64']).columns:
        dataset[column] = dataset[column].astype('int32')
    
    for column in dataset.select_dtypes(include=['float64']).columns:
        dataset[column] = dataset[column].astype('float32')

In [80]:
df_oil.dtypes

date           object
dcoilwtico    float32
dtype: object

In [ ]:
def fillnan(data):
    i=list[set(dftrian['city'])]

In [5]:
def certprocess(data):
    data['date']=pd.to_datetime(data['date'],format='%Y-%m-%d')
    data=data.set_index(['date'])
    
    cols = data.select_dtypes(include=['object']).columns
    data = pd.get_dummies(data, columns=cols, prefix=cols, prefix_sep='-')
    
    return data


### 注意select_dtypes函数，可以挑选出type=object的列